### Guided modes 1 + 1D systems (TE modes)

In [2]:
# Uploaded the different libraries
import numpy as np
import scipy.sparse as sps
from scipy.sparse.linalg import eigs
import matplotlib.pyplot as plt



import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from matplotlib import cm

In [3]:
#1D Example Parameters
grid_size     = 120
number_points = 601
h             = grid_size/(number_points - 1)
lam           = 0.78
k0            = 2*(np.pi/lam)
e_substrate   = 2.25
delta_e       = 1.5e-2
w             = 15.0
xx            = np.linspace( -grid_size/2, grid_size/2, number_points )
prm           = e_substrate + delta_e * np.exp(-(xx/w)**2)

In [4]:
def guided_modes_1DTE(prm, k0, h):
    """Computes the effective permittivity of a TE polarized guided eigenmode.
    All dimensions are in µm.
    Note that modes are filtered to match the requirement that
    their effective permittivity is larger than the substrate (cladding).
    
    Parameters
    ----------
    prm : 1d-array
        Dielectric permittivity in the x-direction
    k0 : float
        Free space wavenumber
    h : float
        Spatial discretization
    
    Returns
    -------
    eff_eps : 1d-array
        Effective permittivity vector of calculated modes
    guided : 2d-array
        Field distributions of the guided eigenmodes
    """
    ###
    # First: initialization of the diagonals that are going to the matrix
    ###
    diagonal_sides = np.full((len(prm)-1),1/(h**2)) # diagonal sides
    diagonal_main =   -2 / (h ** 2) + k0 ** 2 * np.array(prm)

    ###
    # Initialize the main diagonal and add the diagonals to the matrix
    ###
    matrix = (
        np.diag(diagonal_main) +
        np.diag(diagonal_sides, k=1) +
        np.diag(diagonal_sides, k=-1)
    )  # adding the diagonals to matrix using the recomended commad :)
    matrix = matrix * (1/k0**2) # multiply by the common factor

    ###
    # Getting eigenvalues and eigenvectors
    # eigenvalues : effective epsilons 
    # eigenvectos : the different effective fields
    # Initializing first then getting the physical epsilons
    ###

    eigenvalues, eigenvectors = np.linalg.eig(matrix) # obtaining the eigenvalues and eigenvectors of the matrix
    values_interest = (eigenvalues > e_substrate) & (eigenvalues < np.max(prm)) # select the physical epsilons
    eff_eps = eigenvalues[values_interest] # Append the pysical epsilons into the parameter requested
    guided = eigenvectors[:, values_interest] # Append the pysical fields into the parameter requested, here we take all the rows and only select the colums that fullfill our condition
    return eff_eps, guided

eff_eps, guided = guided_modes_1DTE(prm, k0, h)

### Guided modes in 2+1 (=3D) systems (strip waveguide)

These are the values originaly offered to us to try out function we have decided to keep them to try the code and show the different plots.

In [5]:
#2D Example Parameters
grid_size     = 120
number_points = 301
h             = grid_size/(number_points - 1)
lam           = 0.78
k0            = 2*np.pi/lam
e_substrate   = 2.25
delta_e       = 1.5e-2
w             = 15.0
xx            = np.linspace(-grid_size/2-h,grid_size/(2+h),number_points+2)
yy            = np.linspace(-grid_size/2,grid_size/2,number_points)
XX,YY         = np.meshgrid(xx,yy)
prm           = e_substrate + delta_e * np.exp(-(XX**2+YY**2)/w**2)



To observe how different modes propagate within a strip waveguide, one can begin by solving the Helmholtz equation and analyzing the spatial distribution of the wavefunction. In this markdown, we provide an overview of the mathematical steps involved, along with code analogies used to solve the exercise.

During this exercice we have done two assuptions: the first one that there is no z-dependence of the traveling modes and that there is weak guiding, this is
what allows us to use the helmholz equation as the starting point of our approach.

$$
\nabla^2 v(r) + k^2(r, \omega) v(r) = 0 
$$
with
$$
k^2(r,w) = \omega^2/c^2
\epsilon(r,w)
$$

As a first order differential equation one can make the following antsatz that the stationary modes should take:
$$
v(r) = u(x,y)exp(i\beta z)
$$

If one plugs this into the helmholz equation we obtain the following equation, which then is an eigenvalue problem that one can solve and obtain the standing modes that can propagate in the waveguide:
$$
[\frac{\partial^2 }{\partial x^2}+ \frac{\partial^2}{\partial y^2}] u(x,y) + [k^2(x,y, \omega)-\beta ^2(w)]u(x,y)  = 0
$$

As computers cannot work directly with the differential equations, one has to discretice the space in both the x and y direction. Using the finite different method, which then would simplify the above equation into the following:

$$
[\frac{\partial^2 }{\partial x^2}+ \frac{\partial^2}{\partial y^2}] =
\frac{f(x_{j+1},y_k)+f(x_{j-1},y_k)+f(x_{j},y_{k+1})+f(x_j,y_{k-1}) - 4f(x_j,y_{k}) }{h^2}
$$

$$
[\frac{\partial^2 }{\partial x^2}+ \frac{\partial^2}{\partial y^2}] u(x,y) + [k^2(x,y,\omega)-\beta ^2(w)]u(x,y)  = 0
$$

As this is an eigenvalue problem one can rewrite the equation to the following way and this is the start point from which we can take the mathematical implementation:
$$
1/k_0^2[[\frac{\partial^2 }{\partial x^2}+ \frac{\partial^2}{\partial y^2}] u(x,y) + k_{0}^2 \epsilon(x,y,\omega)u(x,y) ] = \epsilon _{eff} u(x,y)
$$
with
$$
k_{0}^2= \frac{w^2 }{c^2}
$$
and with the dispersion relation:
$$
k^2(x,y,\omega)= k_{0}^2\epsilon(x,y,\omega)
$$
and the effective epsilons are then defined as
$$
\epsilon _{eff} = \frac{\beta^2}{k_{0}^2}
$$


As shown above, the differential equation is approximated with a finite difference model, this means that the elements depends on its previous and the fordward,the upward, the downward points when dealing with a 2D domain. A good way to code this is simlpy using a matrix, and because we are solving an eigenvalue problem, solving the eigenvalues of this matrix would give us the solutions we are looking for the effective epsilons.


The fist line in our matrix is the main diagonal, in this case, is when the element gets multiply by itself which in code and math is the following:

math:
$$
- 4\frac{f(x_j,y_{k})}{h^2} + k_{0}^2 \epsilon(x,y,\omega)
$$
Wich in code then can be rewriten as:
 -4.0 *1/(h**2) + k0 ** 2 * np.array(prm_flat)
with np.array(prm_flat) being the epsilons which in our case is a gaussian function 

For the other diagonals are positioned next to this main element, and in the position in which we are jumping into a new row. This means the diagonals corresponding to the adjancent points need to be the upper and lower diagonal to this main diagonal and the ones corresponding to the y positon are where the new row is starting.

As all of them have the same values one can code a simple initialization line and then positon it in the different diagonals.
math:
$$
\frac{f(x_{j+1},y_k)}{h^2}   > diagonal position 1 
$$
$$
\frac{f(x_{j-1},y_k)}{h^2}      > diagonal position -1 
$$
$$
\frac{f(x_{j},y_{k+1})}{h^2}     > diagonal position: nrows 
$$
$$
\frac{f(x_{j},y_{k-1})}{h^2}      > diagonal position: -nrows
$$


Wich in code then can be rewriten as:
np.ones(n)*1/(h**2) 
with n being 
$$
nrows*ncols
$$
To code the positions one can use the follwing  and scipy.sparse.spdiags  which leads to the follwing lines:

data = np.array([diagonal_main, diagonal_sides, diagonal_sides, diagonal_sides,  diagonal_sides]) # collect all diagonals into one requested with the function that we are going to use

diags = np.array([0,-1,1,-nrows, nrows]) # position of diagonals

matrix =sps.spdiags(data, diags, n, n).tocsc() * (1/k0**2) 

As we mentioned this is a eigenvalue problem and we can use the following coding function scipy.sparse.spdiags to get the eigenvalues and eigenvectors which fullfill our helmholz equation which then in code is translated to:
eigenvalues, eigenvectors = eigs(matrix, numb)

as we can obtain a large amount of eigenvalues to reduce the amount of eigenvalues provided the user can define a maximun of eigenvalues with the variable numb.

All the eigenvalues obtained are not physical this is, they are either smaller than the cladding or the maxima of the distribution of the refractive index of our media. Therefore, one has to just select the phsysical solutions of the code with the following lines:

values_interest = (eigenvalues > e_substrate) & (eigenvalues < np.max(prm))  

eff_eps = eigenvalues[values_interest]

guided = eigenvectors[:, values_interest] 

As we are requested to return a 3D array one can rewtite it as follows:
guided = guided.reshape((nrows, ncols, -1), order='F') 






In [ ]:
def guided_modes_2D(prm, k0, h, numb):
    """Computes the effective permittivity of a quasi-TE polarized guided 
    eigenmode. All dimensions are in µm.
    
    Parameters
    ----------
    prm  : 2d-array
        Dielectric permittivity in the xy-plane
    k0 : float
        Free space wavenumber
    h : float
        Spatial discretization
    numb : int
        Number of eigenmodes to be calculated
    
    Returns
    -------
    eff_eps : 1d-array
        Effective permittivity vector of calculated eigenmodes
    guided : 3d-array
        Field distributions of the guided eigenmodes
    """
    ###
    # First: Flatted the 2D array into a column, this is becuase to obtain the different eigenvalues
    # the objective would be multiplying this column with the general matrix to obtain the
    # different eigenvalues thus we use the flatten command in numpy
    ###
    prm_flat = prm.flatten(order='F')

    ## 
    # Get the number of columns and row and the total number as we have to create arrays based on that shape
    ##
    nrows, ncols = prm.shape
    n = nrows*ncols

    ##
    # Creation of the two different diagonals: 
    # The main diagonal similar to what we have before 
    # The side diagonals which would go in the positons adjacnet to the points in the x direction and the 
    # new rows in the y diretion still adjacted to the square of interest
    ##
    diagonal_main = -4.0 *1/(h**2) + k0 ** 2 * np.array(prm_flat)  # values of the main diagonal
    diagonal_sides = np.ones(n)*1/(h**2) # values of the other diagonals
    data = np.array([diagonal_main, diagonal_sides, diagonal_sides, diagonal_sides,  diagonal_sides]) # data containing information diagonals
    diags = np.array([0,-1,1,-nrows, nrows]) # position of diagonals
    matrix =sps.spdiags(data, diags, n, n).tocsc() * (1/k0**2)  # creation of matrix

    ##
    # Once the matrix is generated we obtain the eigenvalues and obtain only the physical ones
    ##
    eigenvalues, eigenvectors = eigs(matrix, numb, which='LR') # get the eigenvalues
    values_interest = (eigenvalues > e_substrate) & (eigenvalues < np.max(prm)) # limitation of the physical eigenvalues
    eff_eps = eigenvalues[values_interest] # Physical eigenvalues
    guided = eigenvectors[:, values_interest] # obtain the fields corresponding to those eigenvalues
    guided = guided.reshape((nrows, ncols, -1), order = "F") # reshape the array to obtain a 3D array
    return eff_eps, guided

Plot the different TEM modes that we have calculated, as one can see the solutions that we obtain are similar to the ones one can find in a textbook.

In [ ]:
eff_eps, guided = guided_modes_2D(prm, k0, h, 40)

# Filtion to plot the different eigenfields
def plot_mode(mode_num):
    plt.figure(figsize=(6,5))
    plt.imshow(np.abs(guided[:, :, mode_num]), cmap='inferno', origin='lower')
    plt.colorbar(label='|Field Amplitude|')
    plt.title(f'Mode {mode_num + 1}')
    plt.xlabel('x grid')
    plt.ylabel('y grid')
    plt.show()

# Cursor to run over the different fields that one has calculated
mode_selector = widgets.IntSlider(
    value=0,
    min=0,
    max=guided.shape[2]-1,
    step=1,
    description='Mode:',
    continuous_update=False
)
#Final function to plot them 
widgets.interact(plot_mode, mode_num=mode_selector)


interactive(children=(IntSlider(value=0, continuous_update=False, description='Mode:', max=39), Output()), _do…

<function __main__.plot_mode(mode_num)>